In [1]:
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import time

In [2]:
total_meetings = requests.get("https://api.openf1.org/v1/meetings").json()
session_dict = {}
lap_dict = {}
for meeting in tqdm(total_meetings):
    mk = meeting['meeting_key']
    session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    while (session_dict[mk].status_code != 200):
        time.sleep(0.5)
        session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    session_dict[mk] = session_dict[mk].json()
    for session in session_dict[mk]:
        if session['session_type'] == 'Race':
            session_key = session['session_key']
            lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            while (lap_dict[mk].status_code != 200):
                time.sleep(0.5)
                lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            lap_dict[mk] = lap_dict[mk].json()
            

100%|██████████| 32/32 [01:33<00:00,  2.92s/it]


In [3]:
for elem in lap_dict.keys():
    if elem not in session_dict.keys(): 
        print(elem in session_dict.keys(), elem)

In [4]:
def get_total_df(lst_dict):
    ret = []
    for k, v in lst_dict.items():
        ret += v
    return pd.DataFrame(ret)

total_laps_df = get_total_df(lap_dict)
# total_laps_df = total_laps_df.loc[:, ["meeting_key", "session_key", "driver_number", "st_speed", "lap_number"]]
total_laps_df.to_csv("laps.csv", index = False)
# total_laps_df.groupby(['meeting_key', 'session_key', 'driver_number']).max().loc[:, "st_speed"]
display(total_laps_df)

,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1141,7953,1,232.0,231.0,252.0,None,NaN,False,NaN,42.414,23.842,"[2048, 2048, 2049, 2049, 2051, 2051, 2049, 204...","[2051, 2049, 2051, 2049, 2051, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 0]",1
1,1141,7953,2,224.0,249.0,269.0,None,NaN,False,NaN,45.330,24.706,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2049]",1
2,1141,7953,4,229.0,236.0,263.0,None,NaN,False,NaN,44.789,24.561,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2049, 2048, 2048]",1
3,1141,7953,10,214.0,234.0,271.0,None,NaN,False,NaN,46.780,25.135,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2048, 2048, 2049, 2048, 0]",1
4,1141,7953,11,230.0,237.0,252.0,None,NaN,False,NaN,43.216,24.069,"[2048, 2048, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2049]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34244,1237,9531,31,258.0,284.0,326.0,2024-06-09T19:48:13.253000+00:00,77.012,False,22.139,24.500,30.373,"[None, 2048, 2048, 2048, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2049]","[2048, 2048, 2048, 0, 0, 0, 0]",70
34245,1237,9531,44,262.0,287.0,334.0,2024-06-09T19:47:50.060000+00:00,74.856,False,21.217,23.745,29.894,"[2051, 2048, 2048, 2048, 2048, 2051]","[2048, 2049, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2064, 2064]",70
34246,1237,9531,63,258.0,284.0,326.0,2024-06-09T19:47:49.468000+00:00,74.895,False,21.203,23.671,30.021,"[2048, 2048, 2051, 2048, 2049, 2048]","[2048, 2048, 2048, 2051, 2051, 2048]","[2048, 2048, 2048, 2048, 2048, 2064, 2064]",70
34247,1237,9531,77,253.0,281.0,302.0,2024-06-09T19:48:21.450000+00:00,79.042,False,22.472,24.991,31.579,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 0, 0, 0, 0, 0, 0]",70


In [19]:
# 문자열을 datetime 객체로 변환하는 함수
def parse_datetime(date_str):
    if '.' in date_str:
        # 소수점이 있는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f%z")
    else:
        # 소수점이 없는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")

total_session_df = get_total_df(session_dict)
total_session_df = total_session_df.loc[:, ["meeting_key", "session_key", "session_type", "date_start"]]
total_session_df['date_start'] = total_session_df['date_start'].map(parse_datetime)
total_session_df

,meeting_key,session_key,session_type,date_start
0,1140,9222,Practice,2023-02-23 07:00:00+00:00
1,1140,7763,Practice,2023-02-24 07:00:00+00:00
2,1140,7764,Practice,2023-02-25 07:00:00+00:00
3,1141,7765,Practice,2023-03-03 11:30:00+00:00
4,1141,7766,Practice,2023-03-03 15:00:00+00:00
...,...,...,...,...
152,1237,9524,Practice,2024-06-07 17:30:00+00:00
153,1237,9525,Practice,2024-06-07 21:00:00+00:00
154,1237,9526,Practice,2024-06-08 16:30:00+00:00
155,1237,9527,Qualifying,2024-06-08 20:00:00+00:00


In [23]:
total_meeting_df = pd.DataFrame(total_meetings)
total_meeting_df = total_meeting_df.loc[:, ["meeting_key", "meeting_name", "country_name", "circuit_short_name", "location", "year"]]
total_meeting_df

,meeting_key,meeting_name,country_name,circuit_short_name,location,year
0,1140,Pre-Season Testing,Bahrain,Sakhir,Sakhir,2023
1,1141,Bahrain Grand Prix,Bahrain,Sakhir,Sakhir,2023
2,1142,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,Jeddah,2023
3,1143,Australian Grand Prix,Australia,Melbourne,Melbourne,2023
4,1207,Azerbaijan Grand Prix,Azerbaijan,Baku,Baku,2023
5,1208,Miami Grand Prix,United States,Miami,Miami,2023
6,1210,Monaco Grand Prix,Monaco,Monte Carlo,Monaco,2023
7,1211,Spanish Grand Prix,Spain,Catalunya,Barcelona,2023
8,1212,Canadian Grand Prix,Canada,Montreal,Montréal,2023
9,1213,Austrian Grand Prix,Austria,Spielberg,Spielberg,2023
